# Spotify Music Popularity Prediction
Este notebook aborda o processo completo de construção de um modelo de Machine Learning para prever a popularidade de músicas no Spotify. O processo inclui a limpeza e tratamento de dados, análise exploratória, treinamento de modelo e avaliação do desempenho.

## Etapa 1: Importação e Limpeza dos Dados
Nesta etapa, os dados de treino e teste são carregados e as colunas com valores nulos são tratadas.

In [3]:
# Importando as bibliotecas necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report

Lê os arquivos necessários para a predição

In [4]:
df_test = pd.read_csv('test.csv')
df_train = pd.read_csv('train.csv')

Preenche linhas nulas com "Desconhecido"

In [5]:
# Altera linhas com valores nulos nas colunas específicas
df_test['artists'] = df_test['artists'].fillna('Desconhecido')
df_test['album_name'] = df_test['album_name'].fillna('Desconhecido')
df_test['track_name'] = df_test['track_name'].fillna('Desconhecido')

## Etapa 2: Separar Features e Alvo
Nesta etapa, as features (variáveis independentes) e o alvo (variável dependente) são separadas para posterior treinamento do modelo.

In [6]:
# Remover colunas irrelevantes para o modelo
df_test_cleaned = df_test.drop(columns=['track_unique_id', 'track_id', 'artists', 'album_name', 'track_name'])
df_train_cleaned = df_train.drop(columns=['track_unique_id', 'track_id', 'artists', 'album_name', 'track_name'])


Trata outliers e remove valores fora da realidade

In [7]:
# Remove músicas com duração inferior a 30 segundos e superior a 10 minutos
df_train_cleaned = df_train_cleaned[(df_train_cleaned['duration_ms'] > 30000) & (df_train_cleaned['duration_ms'] < 600000)]

# Remove músicas com tempo inferior a 0
df_train_cleaned = df_train_cleaned[df_train_cleaned['tempo'] >= 0]

# Remove músicas com volume inferior a -60
df_train_cleaned = df_train_cleaned[df_train_cleaned['loudness'] >= -60]

# Remove músicas com energia inferior a 0
df_train_cleaned = df_train_cleaned[df_train_cleaned['energy'] >= 0]

# Remove músicas com dançabilidade inferior a 0
df_train_cleaned = df_train_cleaned[df_train_cleaned['danceability'] >= 0]

# Remove músicas com valência inferior a 0
df_train_cleaned = df_train_cleaned[df_train_cleaned['valence'] >= 0]

# Remove músicas com acusticidade inferior a 0
df_train_cleaned = df_train_cleaned[df_train_cleaned['acousticness'] >= 0]

# Remove músicas com instrumentalidade inferior a 0
df_train_cleaned = df_train_cleaned[df_train_cleaned['instrumentalness'] >= 0]

# Remove músicas com vivacidade inferior a 0
df_train_cleaned = df_train_cleaned[df_train_cleaned['liveness'] >= 0]

# Remove músicas com falabilidade inferior a 0
df_train_cleaned = df_train_cleaned[df_train_cleaned['speechiness'] >= 0]

# Remove músicas com conteúdo explícito inferior a 0
df_train_cleaned = df_train_cleaned[df_train_cleaned['explicit'] >= 0]

# Remove músicas com modo inferior a 0
df_train_cleaned = df_train_cleaned[df_train_cleaned['mode'] >= 0]

# Remove músicas com tom musical inferior a 0
df_train_cleaned = df_train_cleaned[df_train_cleaned['key'] >= 0]

# Remove músicas com compasso inferior a 0
df_train_cleaned = df_train_cleaned[df_train_cleaned['time_signature'] >= 0]


In [8]:
# Dividir o dataset de treino em X e y
X_train = df_train_cleaned.drop(columns=['popularity_target'])
y_train = df_train_cleaned['popularity_target']

## Etapa 3: Codificação de Variáveis Categóricas
As variáveis categóricas são transformadas em numéricas usando One-Hot Encoding.

In [9]:
# Codificar as variáveis categóricas usando One-Hot Encoding
X_train_encoded = pd.get_dummies(X_train, drop_first=True)
X_test_encoded = pd.get_dummies(df_test_cleaned, drop_first=True)

## Etapa 4: Divisão dos Dados
Dividir os dados em conjunto de treino e teste para posterior avaliação do modelo.

In [10]:
# Alinhar as colunas do conjunto de treino e teste para garantir que as colunas sejam iguais
X_train_encoded, X_test_encoded = X_train_encoded.align(X_test_encoded, join='left', axis=1, fill_value=0)

# Separar dados de treino e validação (opcional, se não estiver usando conjunto de teste separado)
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train_encoded, y_train, test_size=0.2, random_state=42)


## Etapa 5: Treinamento do Modelo
Treinar o modelo de Random Forest com os dados de treino.

In [11]:
# # Definir os hiperparâmetros a serem ajustados
# param_grid = {
#     'n_estimators': [100, 200, 500],            # Número de árvores
#     'max_depth': [10, 20, 30, None],            # Profundidade máxima
#     'min_samples_split': [2, 5, 10],            # Número mínimo de amostras para dividir um nó
#     'min_samples_leaf': [1, 2, 4],              # Número mínimo de amostras em uma folha
#     'max_features': ['sqrt', 'log2'],           # Número de features a considerar por divisão
#     'bootstrap': [True, False]                  # Usar amostras bootstrap ou não
# }

# # Instanciar o modelo com balanceamento de classes
# rf = RandomForestClassifier(class_weight='balanced', random_state=42)

# # Usar RandomizedSearchCV para ajustar os hiperparâmetros
# rf_random = RandomizedSearchCV(estimator=rf, param_distributions=param_grid,
#                                n_iter=100, cv=3, verbose=2, random_state=42, n_jobs=-1, scoring='precision')


# # Ajustar o modelo
# rf_random.fit(X_train_split, y_train_split)

# # Ver os melhores hiperparâmetros encontrados
# print("Melhores Hiperparâmetros:", rf_random.best_params_)

In [12]:
# Melhores Hiperparâmetros: {'n_estimators': 200, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': None, 'bootstrap': False}

rf_random = RandomForestClassifier(n_estimators=200, min_samples_split=2, min_samples_leaf=1, max_features='sqrt', max_depth=None, bootstrap=False)

# Treinamento do modelo
rf_random.fit(X_train_split, y_train_split)


RandomForestClassifier(bootstrap=False, n_estimators=200)

## Etapa 6: Avaliação do Modelo
Fazer previsões no conjunto de teste e avaliar o desempenho do modelo.

In [13]:
# Previsão e avaliação no conjunto de validação
y_pred = rf_random.predict(X_val_split)
print("Relatório de Classificação (Validação):\n", classification_report(y_val_split, y_pred))

# Previsão no conjunto de teste
y_test_pred = rf_random.predict(X_test_encoded)


Relatório de Classificação (Validação):
               precision    recall  f1-score   support

           0       0.85      0.84      0.85      8123
           1       0.84      0.84      0.84      7753

    accuracy                           0.84     15876
   macro avg       0.84      0.84      0.84     15876
weighted avg       0.84      0.84      0.84     15876



Salva as previsões em um arquivo CSV para submissão no Kaggle.

In [14]:
# Salvar previsões no arquivo CSV apenas com os registros que têm previsões
result_df = df_test.copy( )
result_df['popularity_predicted'] = y_test_pred

# Deixa somente as colunas popularity_predicted e track_unique_id
result_df = result_df[['track_unique_id', 'popularity_predicted']]

result_df.to_csv('predicted_results.csv', index=False)
print("Previsões salvas com sucesso.")


Previsões salvas com sucesso.
